<a href="https://colab.research.google.com/github/PAVANIVALETI/colabtools/blob/main/Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import tensorflow as tf
import numpy as np



One_hot encoding

In [10]:
vocab = ["<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY", "ISLAND"]
indices = tf.range(len(vocab), dtype=tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices)
num_oov_buckets = 2
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)

categories = tf.constant(["NEAR BAY", "DESERT", "INLAND", "INLAND"])
cat_indices = table.lookup(categories)
cat_indices

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([3, 5, 1, 1])>

In [4]:
cat_one_hot = tf.one_hot(cat_indices, depth=len(vocab) + num_oov_buckets)
cat_one_hot

<tf.Tensor: shape=(4, 7), dtype=float32, numpy=
array([[0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.]], dtype=float32)>

Embedding Encoding

In [8]:
embedding_dim = 2
embed_init = tf.random.uniform([len(vocab) + num_oov_buckets, embedding_dim])
embedding_matrix = tf.Variable(embed_init)
embedding_matrix


<tf.Variable 'Variable:0' shape=(7, 2) dtype=float32, numpy=
array([[0.5928556 , 0.33880997],
       [0.14440191, 0.20123374],
       [0.72627413, 0.12008822],
       [0.9720473 , 0.5301807 ],
       [0.1821953 , 0.6253544 ],
       [0.5340178 , 0.2086817 ],
       [0.27004576, 0.34849524]], dtype=float32)>

In [11]:
categories = tf.constant(["NEAR BAY", "DESERT", "INLAND", "INLAND"])
cat_indices = table.lookup(categories)
cat_indices
tf.nn.embedding_lookup(embedding_matrix, cat_indices)


<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[0.9720473 , 0.5301807 ],
       [0.5340178 , 0.2086817 ],
       [0.14440191, 0.20123374],
       [0.14440191, 0.20123374]], dtype=float32)>

Creating model using Embedding Encoing 

In [7]:
from tensorflow import keras 
regular_inputs = keras.layers.Input(shape=[8])
categories = keras.layers.Input(shape=[], dtype=tf.string)
cat_indices = keras.layers.Lambda(lambda cats: table.lookup(cats))(categories)
cat_embed = keras.layers.Embedding(input_dim=6, output_dim=2)(cat_indices)
encoded_inputs = keras.layers.concatenate([regular_inputs, cat_embed])
outputs = keras.layers.Dense(1)(encoded_inputs)
model = keras.models.Model(inputs=[regular_inputs, categories],
 outputs=[outputs])
